#### File Setup

In [134]:
#pip install pycountry-convert --quiet
#pip install seaborn  --quiet
#pip install scipy --quiet

In [138]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats

In [179]:
csv_file = "/Users/annaking/Documents/Github/riskychoiceframing_AI/Experiment 1/data/Dissertation - Risky Choice Framing_July 18, 2023_08.57.csv"
df = pd.read_csv(csv_file)
print(df.shape)

(141, 97)


In [180]:
df.columns

Index(['StartDate', 'EndDate', 'Status', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'DistributionChannel',
       'UserLanguage', 'Q_RecaptchaScore', 'Q1. Metadata_Browser',
       'Q1. Metadata_Version', 'Q1. Metadata_Operating System',
       'Q1. Metadata_Resolution', 'Q2. Time_First Click',
       'Q2. Time_Last Click', 'Q2. Time_Page Submit', 'Q2. Time_Click Count',
       'Q3. Consent', 'Q10. GS1, Time_First Click',
       'Q10. GS1, Time_Last Click', 'Q10. GS1, Time_Page Submit',
       'Q10. GS1, Time_Click Count', 'Q12. GS1, Option', 'Q13. GS1, Rating',
       'Q13. GS1, Rationale', 'Q10. GS2, Time_First Click',
       'Q10. GS2, Time_Last Click', 'Q10. GS2, Time_Page Submit',
       'Q10. GS2, Time_Click Count', 'Q12. GS2, Option', 'Q13. GS2, Rating',
       'Q14. GS2, Rationale', 'Q10. GS3, Time_First Click',
       'Q10. GS3, Time_Last Click', 'Q10. GS3, Time_Page Submit',
       'Q10. GS3, Time_Click Count', 'Q12. GS3, Option.', '

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 97 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   StartDate                      141 non-null    object 
 1   EndDate                        141 non-null    object 
 2   Status                         141 non-null    object 
 3   Progress                       141 non-null    int64  
 4   Duration (in seconds)          141 non-null    int64  
 5   Finished                       141 non-null    bool   
 6   RecordedDate                   141 non-null    object 
 7   ResponseId                     141 non-null    object 
 8   DistributionChannel            141 non-null    object 
 9   UserLanguage                   141 non-null    object 
 10  Q_RecaptchaScore               133 non-null    float64
 11  Q1. Metadata_Browser           141 non-null    object 
 12  Q1. Metadata_Version           141 non-null    obj

#### Basic Data Cleaning 

In [145]:
##### Data cleaning to change column types

#drop anyone now sorted into condition 
df_clean = df.drop(df[df['Condition'].isna()].index)

#change data type to int 
df_clean['Condition'] =  df_clean['Condition'].astype('int64')

# create frame column gain or loss condition based on condition value 
df_clean['frame'] = df_clean['Condition'].apply(lambda x: 'gain' if x < 3 else 'loss')

#filter only where progress = 100 
df_clean = df_clean[df_clean['Progress']==100]
print(df_clean.shape)

(126, 98)


In [148]:
## survey timing & outlier removal 

## visual completion times 
#sns.histplot( y='Duration (in seconds)', data =df_clean)
#plt.show()

#look at completion times 
df_clean['Duration (in seconds)'].describe()

def remove_outliers(df, column_name):
    # Calculate Z-scores for the column
    z_scores = stats.zscore(df[column_name])
    threshold = 3 #3 standard deviations 
    # Filter the dataframe to keep only the data within the threshold
    df_clean = df[abs(z_scores) < threshold]

    return df_clean

#apply function to remove outliers 
column_name = 'Duration (in seconds)'
df_clean = remove_outliers(df_clean, column_name) 

df_clean.shape

(116, 98)

In [ ]:
### check for NAs 

##### Mapping Columns & Cleaning

In [194]:
import re
##### Option Column Cleaning
# select option columns 
df_clean['option_selected'] = df_clean[['Q12. GS1, Option','Q12. GS2, Option', 'Q12. GS3, Option.', 'Q12. LS1, Option', 'Q12. LS2, Option', 'Q12. LS3, Option']].fillna(method='ffill', axis=1).iloc[:, -1]
option_columns = ['Q12. GS1, Option','Q12. GS2, Option', 'Q12. GS3, Option.', 'Q12. LS1, Option', 'Q12. LS2, Option', 'Q12. LS3, Option']

#rating column clean & creation 
rating_columns = ['Q13. GS1, Rating','Q13. GS2, Rating', 'Q13. GS3, Rating', 'Q13. LS1, Rating', 'Q13. LS2, Rating', 'Q13. LS3, Rating']
df_clean['rating'] = df_clean[rating_columns].fillna(method='ffill', axis=1).iloc[:, -1]
df_clean['rating_num'] = df_clean['rating'].apply(lambda x: x.split(":")[0])

#rationale column cleaning
rationale_columns = ['Q13. GS1, Rationale','Q14. GS2, Rationale', 'Q14. GS3, Rationle', 'Q14. LS1, Rationale', 'Q14. LS2, Rationale', 'Q14. LS3, Rationale']
df_clean['rationale'] = df_clean[rationale_columns].fillna(method='ffill', axis=1).iloc[:, -1]

##### drop standalone choice, rationale, and rating columns
colu = rationale_columns + option_columns + rating_columns
df_clean = df_clean.drop(colu, axis= 'columns')

In [ ]:
#### Creating columns and grouping for various scenarios:
# dictionary mapping
scenario_dict = {"GS1":"animals_gain", "GS2":"forest_gain", "GS3":"humans_gain", "LS1":"animals_loss", "LS2":"forest_loss", "LS3":"humans_loss"  }

#set columns based on scenario 
def scenario(row):
    for col in selected_columns:
        if pd.notnull(row[col]):
            match = re.search(r'(GS\d+|LS\d+)', col)
            if match:
                return match.group()
    return None
# get scenario from dictionary 
def get_scenario(row):
    value = row['scenario']
    if value in scenario_dict:
        return scenario_dict[value]
    return None
# match scenario to dict
df_clean['scenario'] = df_clean[['Q12. GS1, Option','Q12. GS2, Option', 'Q12. GS3, Option.', 'Q12. LS1, Option', 'Q12. LS2, Option', 'Q12. LS3, Option']].apply(lambda row: scenario(row), axis=1)
df_clean['scenario_mapped'] = df_clean.apply(lambda row: get_scenario(row), axis=1)


In [241]:
### Cleaning and dropping time columns ###
columns = list(df_clean.columns)

###  First click columns
time_click1 = [col for col in columns if 'first click' in col.lower()]
#time first click for the scenario 
time_click1_scn = [string for string in time_click1 if string.startswith('Q10')]
#make as 1 column 
df_clean['scn_t_first_click'] = df_clean[time_click1_scn].fillna(method='ffill', axis=1).iloc[:, -1]

### Last click columns
time_click2 = [col for col in columns if 'last click' in col.lower()]
time_click2_scn = [string for string in time_click2 if string.startswith('Q10')]
df_clean['scn_t_last_click'] = df_clean[time_click2_scn].fillna(method='ffill', axis=1).iloc[:, -1]

### Time Submit Scenario columns
time_submit = [col for col in columns if 'submit' in col.lower()]
time_submit_scn = [string for string in time_submit if string.startswith('Q10')]
df_clean['scn_t_submit'] = df_clean[time_submit_scn].fillna(method='ffill', axis=1).iloc[:, -1]

#click_count1_scn = [string for string in click_count1 if 'dem' in string.lower()]

## drop Q10 time columns 
columns_drop = time_click2_scn + time_click1_scn + time_submit_scn
df_clean = df_clean.drop(columns_drop, axis = 'columns')

##Click count cleaning 
columns = list(df_clean.columns)
## Click Count for Scenario 
click_count1 = [col for col in columns if 'click count' in col.lower()]
#filer only scenario click coun t
click_count_scn = [string for string in click_count1 if string.startswith('Q10')]
#make as 1 column for all scenarios 
df_clean['scn_click_count'] = df_clean[click_count_scn].fillna(method='ffill', axis=1).iloc[:, -1]

## Click Count for Scenario 
click_count = [col for col in columns if 'click count' in col.lower()]
click_count1_dem = [string for string in click_count if 'dem' in string.lower()]
df_clean['dem_click_count'] = df_clean[click_count1_dem].fillna(method='ffill', axis=1).iloc[:, -1]

# drop scenario and dem click count columns 
columns_drop = click_count_scn + click_count1_dem
df_clean = df_clean.drop(columns_drop, axis = 'columns')

### drop metadata or misc columns from analysis 
metadata_or_misc = ['Q1. Metadata_Browser','Q1. Metadata_Version', 'Q1. Metadata_Operating System','Q1. Metadata_Resolution', 'UserLanguage', 'DistributionChannel', 'Status']
df_clean = df_clean.drop(metadata_or_misc, axis = 'columns')


(116, 76)

### Demographics

##### Cleaning 

In [282]:
#rename columns 
df_clean = df_clean.rename(columns={
    'Q5. Age': 'age',
    'Q6. Gender': 'gender',
    'Q6. Gender_4_TEXT': 'gender_text',
    'Q8. Ethnicity': 'ethnicity',
    'Q8. Ethnicity_6_TEXT': 'ethnicity_text',
    'Q9. Education': 'education',
    'Q10. Country': 'country',
    'Q10. Student': 'student',
    'Q91': 'ADP_familiar'
})

In [286]:
###coutry name clean 
import pycountry_convert as pc

#function to rename messy countries
def rename_countries(country):
    if pd.isnull(country):  # Check if the value is nan
        return country
    if country in ['United States of America', 'USA', 'US', 'us', 'usa', 'America', 'America   ', 'the United States', 'The United States',  'United States ', 'Amrican', 'United States', 'Florida', 'New York', 'MA', 'NZ', 'North Carolina', 'Newyork', 'denver']:
        return 'USA'
    elif country in ['United Kingdom', 'United Kingdom ', 'UK', 'England', 'England ', 'Cambridge', 'London', 'uuk', 'the Netherlands']:
        return 'UK'
    elif country in ['China', '中国', 'Chinese ', '美国','加州']:
        return 'China'
    elif country in [ 'New Zealand']:
        return 'New Zealand'
    elif country in ['Paris, France', 'France']:
        return 'France'
    elif country in ['Australia', 'Australia ']:
        return 'Australia'
    elif country in ['pakistan']:
        return 'Pakistan'
    else:
        return country  # return the original name if no match is found

#update country column 
df_clean['country'] = df_clean['country'].apply(rename_countries)
#strip any remaining whitespaces
df_clean['country'] = df_clean['country'].apply(lambda x: x.strip() if isinstance(x, str) else x) #checks if string, then removes spaces 

# Map each country to the continent
def country_to_continent(country_name):
    try:
        if country_name == 'USA':
            country_alpha2 = "US"
        elif country_name == 'UK':
            country_alpha2 = "GB"
        else:
            country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return "Unknown"

df_clean['continent'] = df_clean['country'].apply(country_to_continent)

In [287]:
###### Cat Code Variables #########

# ####Gender: clean gender column into numbers
#0 if female, 1 if male, 2 if non-binary 
df_clean['gender_num'] = df_clean.gender.apply(lambda x: 0 if x == 'Female' else (1 if x == 'Male' else 2))
df_clean.groupby(['gender','gender_num'])['ResponseId'].count()

####Student: clean student column into numbers
#0 if not a student, 1 if a student
df_clean['student_num']= df_clean.student.astype('category').cat.codes
df_clean.groupby(['student','student_num'])['ResponseId'].count()

####Education: clean education column into numbers
#education_num: 0 = Associates, 1 = Bachecholars, 2 = Completed High School, 3 = Grad School, 4 = Prefer Not to Say, 5 = Some high school 
df_clean['education_num']= df_clean.education.astype('category').cat.codes
df_clean.groupby(['education','education_num'])['ResponseId'].count()

#### Ethnicity: clean ethnicity column into numbers
#ethnicity: 0 = African American, 1 = Black/African/Caribbean, 2 = Prefer not to say, 3 = White or Caucasian
df_clean['ethnicity_num']= df_clean.ethnicity.astype('category').cat.codes
df_clean.groupby(['ethnicity','ethnicity_num'])['ResponseId'].count()

#### Continent: clean contintent column into numbers
#continent: 0 = Asia, 1 = Europe, 2 = North America, 3 = Oceania, 4 = South America, 5 = Unknown 
df_clean['contintent_num']= df_clean.continent.astype('category').cat.codes
df_clean.groupby(['continent','contintent_num'])['ResponseId'].count()

#### ADP: lean ADP column into numbers
#ADP: 0 = I'm not sure, 1 = No, 2 = Yes 
df_clean['ADP_num']= df_clean.ADP_familiar.astype('category').cat.codes
df_clean.groupby(['ADP_familiar','ADP_num'])['ResponseId'].count()

#### Age: clean age column into numbers
#age: {'18 - 24 years old': 0, '25 - 34 years old': 1, '35 - 44 years old': 2, '45 - 54 years old': 3, '55 - 64 years old': 4}
age_order = ['18 - 24 years old', '25 - 34 years old', '35 - 44 years old', '45 - 54 years old', '55 - 64 years old']
age_mapping = {age: i for i, age in enumerate(age_order)}
df_clean['age_num'] = df_clean['age'].map(age_mapping) #maps dict 


In [315]:
## Check for NA values 
#print(df_clean[['age_num','age']].isna().value_counts())
#print(df_clean['education'].isna().value_counts())
#print('\n\n',df_clean['ADP_num'].isna().value_counts())
#df_clean['ADP_familiar'].isna().count()

print(f"""school NA: {df_clean['student'].isna().sum()}""")
print(f"""gender NA: {df_clean['gender'].isna().sum()}""")
print(f"""education NA: {df_clean['education'].isna().sum()}""")
print(f"""ADP NA: {df_clean['ADP_familiar'].isna().sum()}""")


school NA: 1
gender NA: 1
education NA: 1
ADP NA: 4


#### Demographic Insights 

Defintions
- gender_num: 0 = female, 1 = male, 2 = other
- student_num: 0 = not a studnet, 1 = a student, -1 = NA
- education_num: 0 = Associates, 1 = Bachecholars, 2 = Completed High School, 3 = Grad School, 4 = Prefer Not to Say, 5 = Some high school 
- ethnicity: 0 = African American, 1 = Black/African/Caribbean, 2 = Prefer not to say, 3 = White or Caucasian
- continent: 0 = Asia, 1 = Europe, 2 = North America, 3 = Oceania, 4 = South America, 5 = Unknown 
- ADP: 0 = I'm not sure, 1 = No, 2 = Yes 
- age: {'18 - 24 years old': 0, '25 - 34 years old': 1, '35 - 44 years old': 2, '45 - 54 years old': 3, '55 - 64 years old': 4}

#### Participant Demographic Insights 

In [102]:
#Participant Demographic Insights 
demographics =   [ 'ResponseId','gender_num', 'gender', 'student_num', 'student','education_num', 'education','ethnicity_num','ethnicity', 'contintent_num', 'continent','age_num', 'age', 'ADP_num', 'ADP_familiar'] 
df_demographics = df_clean[demographics]

#ethnicity
print(df_demographics.ethnicity.value_counts(normalize=True))

ethnicity
White or Caucasian         0.709677
Asian                      0.153226
Black/African/Caribbean    0.129032
Prefer not to say          0.008065
Name: proportion, dtype: float64

In [104]:
##Demographic Insights 
#df_clean.education.value_counts(normalize = True)
#df_clean.groupby(['education_num','frame']).ResponseId.agg(['count', 'sum', 'mean','median'])
#ed = df_clean.groupby(['education_num','frame']).ResponseId.count().reset_index()
#pivoted = ed.pivot(index='education_num', columns='frame', values='ResponseId')
#df_clean[['education_num','frame']].value_counts(normalize =True)

#df_clean.groupby('continent').ResponseId.count()

education
Bachelor's degree                                                   0.379032
Graduate or professional degree (MA, MS, MBA, PHd, JD, MD, etc.)    0.241935
Associates or technical degree                                      0.209677
Completed high school / secondary school                            0.120968
Some high school / secondary school or less                         0.040323
Prefer not to say                                                   0.008065
Name: proportion, dtype: float64

In [107]:
gen_cross_tab = pd.crosstab(df_clean['gender'], df_clean['frame'])
cross_tab

frame,gain,loss
gender,,
Female,28,25
Male,28,38
Non-binary / third gender,2,1
Prefer not to say,1,0
Prefer to self-describe,1,0


In [ ]:
df_clean.groupby(['age', 'age_num'])['ResponseId'].count()

In [42]:
#summary insights 
df_clean.groupby(['frame','scenario','scenario_mapped','option_selected']).ResponseId.count()

#% in each condition (gain or loss)
df_clean['frame'].value_counts()


frame  scenario  scenario_mapped  option_selected
gain   GS1       animals_gain     Proposal A         16
                                  Proposal B          7
       GS2       forest_gain      Proposal A         20
                                  Proposal B          3
       GS3       humans_gain      Proposal A         12
                                  Proposal B          4
loss   LS1       animals_loss     Proposal A         13
                                  Proposal B         10
       LS2       forest_loss      Proposal A         16
                                  Proposal B          5
       LS3       humans_loss      Proposal A         14
                                  Proposal B          9
Name: ResponseId, dtype: int64

In [ ]:
#ADP question 
df_clean['Q91'].value_counts()